In [ ]:
"""Kleine todo:
    -goed kijken co2 data zweden want die is wel erg laag
    -kaart animatie groter krijgen
    -misschien iets doen met gdp / capita ?
    -toevoegen van hoofd- en deelvragen
    - opschonen (er wordt nog vel onzin dingen geprint)
    - misschien ook een kaart maken van zev per land en vergelijken met de gdp / capita kaart (deelvraag 3)"""

# Importing the necessary libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [350]:
# Load database with number of passenger vehicles per country per year
database_1 = pd.read_csv('data/road_eqs_carpda_page_linear.csv')

# Load database with number of zero emmision vehicles per country per year
database_2 = pd.read_csv('data/road_eqs_zev_page_linear.csv')

# Load database with emmisions per country per year
database_3 = pd.read_csv('data/sdg_13_10_page_linear_2.csv')

# Load database with populatoins per country per year
database_4 = pd.read_csv('data/population_1january.csv')

In [351]:
database_4.columns

database_4 = database_4[['geo','TIME_PERIOD', 'OBS_VALUE']]
database_4 = database_4.rename(columns={'OBS_VALUE':'population'})


In [352]:
# Store the unique countries from each database in sets
set1 = set(database_1.geo.unique())
set2 = set(database_2.geo.unique())
set3 = set(database_3.geo.unique())
set4 = set(database_4.geo.unique())

In [353]:
# Identify which countries are in all three databases
countries = set1.intersection(set2, set3, set4)

In [354]:
joined_df_1 = pd.merge(database_1, database_2, how='right', on=['geo', 'TIME_PERIOD'])
joined_df_2 = pd.merge(joined_df_1, database_3, how='left', on=['geo', 'TIME_PERIOD'])
joined_df = pd.merge(joined_df_2, database_4, how='left', on=['geo', 'TIME_PERIOD'])

In [355]:
# Rename columns
joined_df = joined_df.rename(columns={'OBS_VALUE_y':'amount_zev', 'OBS_VALUE' : 'emission_per_capita', 'OBS_VALUE_x' : 'amount_passenger_cars', 'TIME_PERIOD' : 'time_period'})
#joined_df['amount_zev'].head()

In [356]:
# Transforming the abbrevation to the full country names
countries_map = {'AT' : 'Austria',
 'BE' : 'Belgium',
 'BG' : 'Bulgaria',
 'CY' : 'Cyprus',
 'CZ' : 'Czech Republic',
 'DE' : 'Germany',
 'DK': 'Denmark',
 'EE' : 'Estonia',
 'EL' : 'Greece',
 'ES' : 'Spain',
 'EU27_2020' : 'European Union',
 'FI' : 'Finland',
 'FR' : 'France',
 'HR' : 'Croatia',
 'HU' : 'Hungary',
 'IE' :'Ireland',
 'IT' : 'Italy',
 'LT' : 'Lithuania',
 'LU' : 'Luxembourg',
 'LV' : 'Latvia',
 'MT' : 'Malta',
 'NL' : 'Netherlands',
 'PL' : 'Poland',
 'PT': 'Portugal',
 'RO' : 'Romania',
 'SE' : 'Sweden',
 'SI' : 'Slovenia',
 'SK' : 'Slovakia'}

joined_df['geo'] = joined_df['geo'].map(countries_map)

In [ ]:
# Construct new columns with share of zero emmission vehicles and amount of emmission vehicles
joined_df['share_zev'] = joined_df['amount_zev'] / joined_df['amount_passenger_cars']
joined_df['amount_ev'] = joined_df['amount_passenger_cars'] - joined_df['amount_zev']
joined_df['zev_per_capita'] = joined_df['amount_zev'] / joined_df['population']
joined_df['ev_per_capita'] = joined_df['amount_ev'] / joined_df['population']
joined_df['total_emissions'] = joined_df['emission_per_capita'] * joined_df['population']


In [358]:
# Select only the relevant columns from the merged dataset
joined_df = joined_df[['geo', 'time_period', 'amount_passenger_cars', 'amount_zev', 'amount_ev', 'share_zev', 'emission_per_capita', 'zev_per_capita', 'ev_per_capita', 'total_emissions']]
joined_df

,geo,time_period,amount_passenger_cars,amount_zev,amount_ev,share_zev,emission_per_capita,zev_per_capita,ev_per_capita,total_emissions
0,Austria,2014,4694921,3389,4691532,0.000722,8.4,0.000398,0.551440,71465402.4
1,Austria,2015,4748048,5038,4743010,0.001061,8.7,0.000587,0.552481,74688856.2
2,Austria,2016,4821557,9086,4812471,0.001884,8.6,0.001044,0.553128,74824050.6
3,Austria,2017,4898578,14637,4883941,0.002988,9.3,0.001668,0.556710,81587644.5
4,Austria,2018,4978852,20855,4957997,0.004189,9.8,0.002364,0.561987,86458216.6
...,...,...,...,...,...,...,...,...,...,...
275,Slovakia,2019,2393577,956,2392621,0.000399,6.4,0.000175,0.438979,34882694.4
276,Slovakia,2020,2439986,1863,2438123,0.000764,5.5,0.000341,0.446717,30018301.5
277,Slovakia,2021,2493183,3001,2490182,0.001204,6.3,0.000550,0.456096,34396620.3
278,Slovakia,2022,2555491,4531,2550960,0.001773,5.5,0.000834,0.469383,29890916.0


In [ ]:
joined_df = joined_df.sort_values(by=['geo', 'time_period'])

# Obtain difference with previous year for all variables for each country
joined_df['zev_diff'] = joined_df.groupby('geo')['amount_zev'].transform(lambda x: x - x.shift(1))
joined_df['ev_diff'] = joined_df.groupby('geo')['amount_ev'].transform(lambda x: x - x.shift(1))
joined_df['emmission_diff'] = joined_df.groupby('geo')['total_emissions'].transform(lambda x: x - x.shift(1))
joined_df['all_cars_diff'] = joined_df.groupby('geo')['amount_passenger_cars'].transform(lambda x: x - x.shift(1))

# Aurora's normalised data:
joined_df['zev_norm'] = joined_df.groupby('geo')['amount_zev'].transform(lambda x: (x - x.shift(1))/x.shift(1))
joined_df['ev_norm'] = joined_df.groupby('geo')['amount_ev'].transform(lambda x: (x - x.shift(1))/x.shift(1))
joined_df['emmission_norm'] = joined_df.groupby('geo')['total_emissions'].transform(lambda x: (x - x.shift(1))/x.shift(1))
joined_df['all_cars_norm'] = joined_df.groupby('geo')['amount_passenger_cars'].transform(lambda x: (x - x.shift(1))/x.shift(1))

In [360]:
joined_df = joined_df[~(joined_df['geo'] == 'European Union')]

In [361]:
joined_df

,geo,time_period,amount_passenger_cars,amount_zev,amount_ev,share_zev,emission_per_capita,zev_per_capita,ev_per_capita,total_emissions,zev_diff,ev_diff,emmission_diff,all_cars_diff
0,Austria,2014,4694921,3389,4691532,0.000722,8.4,0.000398,0.551440,71465402.4,NaN,NaN,NaN,NaN
1,Austria,2015,4748048,5038,4743010,0.001061,8.7,0.000587,0.552481,74688856.2,1649.0,51478.0,3223453.8,53127.0
2,Austria,2016,4821557,9086,4812471,0.001884,8.6,0.001044,0.553128,74824050.6,4048.0,69461.0,135194.4,73509.0
3,Austria,2017,4898578,14637,4883941,0.002988,9.3,0.001668,0.556710,81587644.5,5551.0,71470.0,6763593.9,77021.0
4,Austria,2018,4978852,20855,4957997,0.004189,9.8,0.002364,0.561987,86458216.6,6218.0,74056.0,4870572.1,80274.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,Sweden,2019,4887116,30382,4856734,0.006217,1.2,0.002970,0.474745,12276222.0,13677.0,3460.0,-3916165.2,17137.0
256,Sweden,2020,4943293,55829,4887464,0.011294,0.4,0.005406,0.473243,4131035.6,25447.0,30730.0,-8145186.4,56177.0
257,Sweden,2021,4985979,110221,4875758,0.022106,0.5,0.010619,0.469758,5189647.5,54392.0,-11706.0,1058611.9,42686.0
258,Sweden,2022,4979761,197751,4782010,0.039711,0.6,0.018919,0.457507,6271395.6,87530.0,-93748.0,1081748.1,-6218.0


In [362]:
# Check whether there are missing values
rows_with_na = joined_df[joined_df.isna().any(axis=1)]

# Deelvraag 1

In [363]:
# Group databases by country
groups = joined_df.groupby('geo')

In [ ]:
df_cleaned = joined_df[['ev_diff', 'emmission_diff', 'geo', 'time_period', 'all_cars_diff', 'zev_diff', 'ev_norm', 'zev_norm', 'emmission_norm', 'all_cars_norm']].dropna()

# Convert to numeric in case they are not
df_cleaned['ev_diff'] = pd.to_numeric(df_cleaned['ev_diff'], errors='coerce')
df_cleaned['emmission_diff'] = pd.to_numeric(df_cleaned['emmission_diff'], errors='coerce')
df_cleaned['all_cars_diff'] = pd.to_numeric(df_cleaned['all_cars_diff'], errors='coerce')

# Drop any remaining NaN values after conversion
df_cleaned = df_cleaned.dropna()


In [365]:
df_cleaned.head()

,ev_diff,emmission_diff,geo,time_period,all_cars_diff,zev_diff
1,51478.0,3223453.8,Austria,2015,53127.0,1649.0
2,69461.0,135194.4,Austria,2016,73509.0,4048.0
3,71470.0,6763593.9,Austria,2017,77021.0,5551.0
4,74056.0,4870572.1,Austria,2018,80274.0,6218.0
5,51987.0,-1413976.6,Austria,2019,60696.0,8709.0


In [375]:
fig = px.scatter(
    df_cleaned,
    x='all_cars_diff',
    y='emmission_diff',
    color='geo',
    symbol='time_period',
    labels={'ev_diff': 'EV Diff', 'emission_diff': 'Emission Diff'},
    trendline='ols',
    title="Scatter Plot of EV Diff vs Emission Diff",
    trendline_scope='overall'
)

fig.show()

In [376]:
fig = px.scatter(
    df_cleaned,
    x='ev_diff',
    y='emmission_diff',
    color='geo',
    symbol='time_period',
    labels={'ev_diff': 'EV Diff', 'emission_diff': 'Emission Diff'},
    #trendline='ols',
    title="Scatter Plot of EV Diff vs Emission Diff",
    trendline_scope='overall'
)

fig.show()

# Deelvraag 2

In [ ]:
fig = px.scatter(
    df_cleaned,
    x='zev_diff',
    y='ev_diff',
    color='geo',
    symbol='time_period',
    labels={'ev_diff': 'EV Diff', 'zev_diff': 'ZEV Diff'},
    trendline='ols',
    title="OLD - Scatter Plot of EV Diff vs ZEV Diff",
    trendline_scope='overall'
)

fig.show()

## Redone plot

In [ ]:
# #Aurora's redone plot:

#dit oude plot hierboven klopt niet, het moeten groeifactoren zijn
#we need to normalise the data, as now the outliers (eg Germany) mess up the data. This normalisation I have done as a percentage 
#change: so percentage change of cars increase per year per country. This way, the fact that eg Germany produces an absolute bigger number of cars doesnt mess up the trend.

# for normalised data see above. I've called the variables _norm

fig = px.scatter(
    df_cleaned,
    x='zev_norm',
    y='ev_norm',
    color='geo',
    symbol='time_period',
    labels={'ev_norm': 'EV percent difference', 'zev_norm': 'ZEV percent difference'},
    trendline='ols',
    title="CORRECTED - Scatter Plot of EV vs ZEV Percent Difference",
    trendline_scope='overall'
)

fig.show()

## Text for report subquestion 2:

# 2. Does an increase in  zero emission vehicles lead to a decrease in emission vehicles?
In this question it will be looked at whether a decrease in emission vehicles actually causes an increase in
zero emission vehicles. 

If this is indeed the case, then the main research question is one step closer to being answered as a decrease in EVs due to an increase in ZEVs would imply an emission reduction that can be traced back to the rise in ZEVS.

### Scatterplot Analysis
In the scatter plot the percentage difference of emission-vehicles versus non-emission vehicles in different EU countries over the years is shown. This percentage difference is a relative number, characterised by the percentage increase in the respective type of vehicle expressed as a growth factor. 

As can be seen from this scatterplot, a negative trend line can be found, implying that the fraction of emission vehicles over the non emission vehicles is decreasing. Thus, there are more non-emission vehicles per emission vehicle.

### Notable Outliers
However, there are a few notable outliers. Some of these inlcude Romania, Poland and Bulgaria. Zooming in on one of these outliers, the Romania 2018 data point in the top left of the scatterplot, it can be seen that in that year the ZEV percentage difference was - 0.9, thus implying a 90% decrease in amount of ZEV compared to the year before (2017). This seems to be quite high, as it would implt Romania got rid of 90% of its ZEVs in that year.

Thus, these outliers need to be further explored, as this is done in subquestion 3, where a timeplot is made in order to investigate the behaviour of the outliers Romania, Poland and Bulgaria.

### Conclusion
Thus it can be concluded that there is a slight negative trend to be seen between EVs and ZEVs, thus implying that a decrease in EVs is connected to an increase in ZEVs.

# Extra plot (Voor de inleiding misschien)

In [382]:
# Filtering out the NaNs of emmission data
filtered_df = joined_df[joined_df['time_period'] != 2023]

#  Making a map animation wich shows the emmissions per EU-country over time
fig = px.choropleth(filtered_df,
                    locations='geo',
                    locationmode='country names',
                    color='emission_per_capita',
                    color_continuous_scale=[[0, "green"], [0.5, "orange"], [1, "red"]],
                    scope='europe',
                    title='Emissions per EU-country over time',
                    animation_frame='time_period',
                    range_color=[0, 16]
                   )

fig.update_layout(
    width=1000,
    height=600,
    margin={"r":0,"t":50,"l":0,"b":0},
    title_x=0.5
    )

fig.show()

In [374]:
joined_df[joined_df['geo'] == 'Sweden']

,geo,time_period,amount_passenger_cars,amount_zev,amount_ev,share_zev,emission_per_capita,zev_per_capita,ev_per_capita,total_emissions,zev_diff,ev_diff,emmission_diff,all_cars_diff
250,Sweden,2014,4584711,2175,4582536,0.000474,0.1,0.000226,0.475127,964486.4,NaN,NaN,NaN,NaN
251,Sweden,2015,4668262,4769,4663493,0.001022,0.3,0.000489,0.478437,2924206.5,2594.0,80957.0,1959720.1,83551.0
252,Sweden,2016,4767262,7555,4759707,0.001585,0.5,0.000767,0.483169,4925508.5,2786.0,96214.0,2001302.0,99000.0
253,Sweden,2017,4844823,11068,4833755,0.002285,1.0,0.001107,0.483610,9995153.0,3513.0,74048.0,5069644.5,77561.0
254,Sweden,2018,4869979,16705,4853274,0.003430,1.6,0.001651,0.479561,16192387.2,5637.0,19519.0,6197234.2,25156.0
255,Sweden,2019,4887116,30382,4856734,0.006217,1.2,0.002970,0.474745,12276222.0,13677.0,3460.0,-3916165.2,17137.0
256,Sweden,2020,4943293,55829,4887464,0.011294,0.4,0.005406,0.473243,4131035.6,25447.0,30730.0,-8145186.4,56177.0
257,Sweden,2021,4985979,110221,4875758,0.022106,0.5,0.010619,0.469758,5189647.5,54392.0,-11706.0,1058611.9,42686.0
258,Sweden,2022,4979761,197751,4782010,0.039711,0.6,0.018919,0.457507,6271395.6,87530.0,-93748.0,1081748.1,-6218.0
259,Sweden,2023,4976366,291716,4684650,0.058620,NaN,0.027726,0.445243,NaN,93965.0,-97360.0,NaN,-3395.0
